Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
        
Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI environment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.

In [2]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [3]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [4]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    
    # Start with a random value function (All 0)
    V = np.zeros(env.nS)
    
    while True:
        delta = 0
        # Full backup for for each state
        for s in range(env.nS):
            v = 0
            # Possible next actions
            for a, action_prob in enumerate(policy[s]):
                # Possible next states for each action
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # Change in value function
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once value function changes below threshold
        if delta < theta:
            break
    return np.array(V)

In [ ]:
def policy_improvement(env,policy_eval_fn=policy_eval,discount_factor=1.0):
    def one_step_lookahead(state,V):
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward+discount_factor*V[next_state])
        return A
    #Random Policy
    policy  = np.ones([env.nS,env.nA]) / env.nA
    
    while True:
        #Evaluate current policy
        V = policy_eval_fn(policy,env,discount_factor)
        
        policy_stable = True
        
        #For each state
        for s in range(env.nS):
            #Best action under current policy
            chosen_a = np.argmax(policy[s])
            action_values = one_step_lookahead(s,V)
            best_a = np.argmax(action_values)
            
            #Update using greedy policy
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]
            
        if policy_